In [ ]:
import numpy as np
import torch
from lib.models.models import UniformRate, UniformVariantRate

from lib.utils import utils
import torch.nn.functional as F
from lib.networks.unet import UNet
from lib.networks.hollow_networks import BidirectionalTransformer
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import lib.utils.bookkeeping as bookkeeping
from tqdm import tqdm
import matplotlib.pyplot as plt
import ssl
import os
import jax
import jax.numpy as jnp
ssl._create_default_https_context = ssl._create_unverified_context
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import time
from torch.utils.data import DataLoader
import lib.sampling.sampling_utils as sampling_utils



In [7]:
D = 32
ddim = D
B = 10
bsize = B
S = 2
device = 'cuda'
vocab_size = S
#mask = jnp.eye(ddim, dtype=jnp.int32).repeat(bsize * vocab_size, axis=0)
#print(mask, mask.shape)
mask = torch.eye(D, device=device, dtype=torch.int32).repeat_interleave(B * S, 0)
print("mask", mask, mask.shape)
min_time = 0.01
ts = torch.rand((B,), device=device) * (1.0 - min_time) + min_time
xt = torch.randint(low=0, high=S, size=(B, D), device=device)

xrep = torch.tile(xt, (D * S, 1))
print("xrep", xrep, xrep.shape)
candidate = torch.arange(S, device=device).repeat_interleave(B, 0)
print("candidate", candidate, candidate.shape)
candidate = torch.tile(candidate.unsqueeze(1), ((D, 1)))
print("candidate 2", candidate, candidate.shape)
xall = mask * candidate + (1 - mask) * xrep
print("xall", xall, xall.shape)
t = torch.tile(ts, (D * S,))
print("t", t, t.shape)

mask tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0', dtype=torch.int32) torch.Size([640, 32])
xrep tensor([[1, 0, 1,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 1,  ..., 0, 1, 0],
        [1, 1, 1,  ..., 0, 1, 0],
        [1, 1, 0,  ..., 1, 1, 1]], device='cuda:0') torch.Size([640, 32])
candidate tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0') torch.Size([20])
candidate 2 tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],


In [6]:
D = 32
ddim = D
B = 10
bsize = B
S = 2
device = 'cuda'
vocab_size = S
rng = jax.random.PRNGKey(0)
t = jax.random.uniform(rng, (bsize,))
xt = jax.random.randint(rng, (B, D), minval=0,
                        maxval=S, dtype=jnp.int32)
mask = jnp.eye(ddim, dtype=jnp.int32).repeat(bsize * vocab_size, axis=0)
print("mask", mask, mask.shape)
xrep = jnp.tile(xt, (ddim * vocab_size, 1))
print("xrep", xrep, xrep.shape)
candidate = jnp.arange(vocab_size).repeat(bsize, axis=0)
print("candidate", candidate, candidate.shape)
candidate = jnp.tile(jnp.expand_dims(candidate, axis=1), ((ddim, 1)))
print("candidate 2", candidate, candidate.shape)
xall = mask * candidate + (1 - mask) * xrep
print("xall", xall, xall.shape)
t = jnp.tile(t, (ddim * vocab_size,))
print("t", t, t.shape)

mask [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]] (640, 32)
xrep [[0 1 1 ... 1 1 1]
 [0 1 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]] (640, 32)
candidate [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1] (20,)
candidate 2 [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]


In [ ]:
from lib.utils import utils
import torch
import jax.numpy as jnp
S = 2
N = 5
D = 32
device = 'cpu'
xt = torch.randint(low=0, high=S, size=(N, D), device=device)
choices = jnp.expand_dims(jnp.arange(S, dtype=jnp.int32),
                            axis=list(range(xt.ndim)))
print(choices, choices.shape)
choices = utils.expand_dims(
    torch.arange(S, dtype=torch.int32), axis=list(range(xt.ndim))
)
print(choices, choices.shape)

In [ ]:
script_dir = '/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/TAUnSDDM/'
save_location = os.path.join(script_dir, 'SavedModels/Synthetic/')
save_location_png = os.path.join(save_location, 'PNGs/')
dataset_location = os.path.join(script_dir, 'lib/datasets/Synthetic/data_2spirals.npy')


cfg = get_config()

device = cfg.device
dataset = dataset_utils.get_dataset(cfg, device, dataset_location)
dataloader = DataLoader(dataset,
    batch_size=1000,
    shuffle=cfg.data.shuffle)

In [ ]:
bm, inv_bm = dataset_utils.get_binmap(cfg.concat_dim, cfg.data.binmode)
for samples in dataloader:
    samples = samples.numpy()
    samples = dataset_utils.bin2float(
        samples.astype(np.int32), inv_bm, cfg.concat_dim, cfg.data.int_scale
    )

    saving_plot_path = os.path.join(
        save_location_png,
        f"{cfg.loss.name}test_{cfg.sampler.name}{cfg.sampler.num_steps}.pdf",
    )
    dataset_utils.plot_samples(
        samples, saving_plot_path, im_size=4.1, im_fmt="pdf"
    )
    break

In [ ]:
i = 0
x_mean = 0 # 127.5
x_std = 0 # 73.7

for minibatch in dataloader:
    
    B = minibatch.shape[0]
    # hollow xt, t, l_all, l_xt geht rein
    device = cfg.device
    ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.0
    ts = torch.ones((B, ), device=device) * 0
    #print(ts)
    #ts = torch.ones((B, )) 
    qt0 = model.transition(ts)  # (B, S, S)

    # rate = model.rate(ts)  # (B, S, S)

    b = utils.expand_dims(torch.arange(B), (tuple(range(1, minibatch.dim()))))
    qt0 = qt0[b, minibatch.long()]

    # log loss
    log_qt0 = torch.where(qt0 <= 0.0, -1e9, torch.log(qt0))
    xt = torch.distributions.categorical.Categorical(logits=log_qt0).sample()
    xt = xt.numpy()
    #print(type(xt))
    samples = dataset_utils.bin2float(
        xt.astype(np.int32), inv_bm, cfg.concat_dim, cfg.data.int_scale
    )

    saving_plot_path = os.path.join(
        save_location_png,
        f"{cfg.loss.name}noisy_{cfg.sampler.name}{cfg.sampler.num_steps}.png",
    )
    dataset_utils.plot_samples(
        samples, saving_plot_path, im_size=cfg.data.plot_size, im_fmt="png"
    )

    break
xt = torch.randint(low=0, high=2, size=(1000, 32), device=device)
xt = xt.numpy()
    #print(type(xt))
samples = dataset_utils.bin2float(
    xt.astype(np.int32), inv_bm, cfg.concat_dim, cfg.data.int_scale
)

saving_plot_path = os.path.join(
    save_location_png,
    f"{cfg.loss.name}realnoisy_{cfg.sampler.name}{cfg.sampler.num_steps}.png",
)
dataset_utils.plot_samples(
    samples, saving_plot_path, im_size=cfg.data.plot_size, im_fmt="png"
)

In [ ]:
import torch

device = 'cpu'  # Beispielgerät
num_steps = 100
max_time = 1
min_time = 0.01
time_dilation = 2

# Szenario 1: time_dilation_start_time ist None
time_steps = torch.linspace(
    max_time, min_time, num_steps * time_dilation + 1, device=device
)
step_sizes = time_steps[:-1] - time_steps[1:]
time_steps = time_steps[:-1]

print("Szenario 1 - Time Steps:", time_steps, time_steps.shape)
print("Szenario 1 - Step Sizes:", step_sizes, step_sizes.shape)

# Szenario 2: time_dilation_start_time ist 0.1
time_dilation_start_time = 0.1
num_steps_first = round(100 * (max_time - time_dilation_start_time) / max_time) + 1
num_steps_second = round(100 * (time_dilation_start_time - min_time) / max_time) * time_dilation + 1

time_steps_first = torch.linspace(max_time, time_dilation_start_time, num_steps_first, device=device)[:-1]
time_steps_second = torch.linspace(time_dilation_start_time, min_time, num_steps_second, device=device)
time_steps = torch.cat([time_steps_first, time_steps_second])

step_sizes = time_steps[:-1] - time_steps[1:]
time_steps = time_steps[:-1]

print("Szenario 2 - Time Steps:", time_steps, time_steps.shape)
print("Szenario 2 - Step Sizes:", step_sizes, step_sizes.shape)
